In [1]:
import pandas as pd
import pickle
import psycopg2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
from ortools.linear_solver import pywraplp
import pandas as pd

from scipy.optimize import linprog


matplotlib.use('TkAgg')

In [7]:
# Load the model
with open('random_forest_model.pkl', 'rb') as file:
    model = pickle.load(file)
print('DONE')

DONE


In [8]:

user_profile = {
    'professional_review':0, 
    'vibe_review': 0, 
    'redness_review': 0, 
    'dry_review': 0, 
    'light_coverage_review': 1, 
    'young_review': 1, 
    'mother_review': 1, 
    'skin_concerns_review': 1, 
    'white_review': 0, 
    'tan_review': 1, 
    'acne_review': 1 ,
    'black_review':0 , 
    'comfortable_wear_review': 0, 
    'coverage_review': 0, 
    'medium_coverage_review': 0, 
    'full_coverage_review': 0, 
    'easy_use_review': 1, 
    'wrinkles_review': 0
}
product_data = pd.read_csv('reddit_product_embeddings.csv')
product_copy = product_data.copy()
product_copy = product_copy.drop(columns = ['product_link_id', 'overall_product_rating'])
le = LabelEncoder()
product_copy['category'] = le.fit_transform(product_copy['category'])

print(product_copy.columns)






Index(['young', 'mother', 'professional', 'vibe', 'acne', 'dry', 'wrinkles',
       'black', 'white', 'tan', 'redness', 'light_coverage', 'medium_coverage',
       'full_coverage', 'sheer_finish', 'glowy_finish', 'matte_finish',
       'expensive', 'inexpensive', 'skin_concerns', 'comfortable_wear',
       'easy_use', 'pilling', 'shade_range', 'category', 'num_shades',
       'num_reviews'],
      dtype='object')


In [9]:
for col, val in user_profile.items(): 
    product_copy[col] = user_profile[col]



In [10]:

feature_names = model.feature_names_in_  # This assumes scikit-learn's RandomForest model
print(feature_names)
# Reorder the DataFrame to match the model's expected feature order
product_copy = product_copy[feature_names]
predictions = model.predict_proba(product_copy)
product_copy['predicted_score'] = predictions[:, 0] + 2* predictions[:, 1] + 3*predictions[:, 2] + 4*predictions[:, 3] + 5*predictions[:, 4]
#product_copy['predicted_score'] = predictions[:, 4] - predictions[:, 1]
print(predictions[0, :])
product_copy['product_link_id'] = product_data['product_link_id']
top_products = product_copy.sort_values(by='predicted_score', ascending=False)
# Output the top products
top_n = 10  # Number of top products to display
print(top_products[['product_link_id', 'predicted_score']].head(top_n))

['young_review' 'mother_review' 'professional_review' 'vibe_review'
 'acne_review' 'dry_review' 'wrinkles_review' 'black_review'
 'white_review' 'tan_review' 'redness_review' 'light_coverage_review'
 'medium_coverage_review' 'full_coverage_review' 'skin_concerns_review'
 'comfortable_wear_review' 'easy_use_review' 'category' 'num_shades'
 'num_reviews' 'young' 'mother' 'professional' 'vibe' 'acne' 'dry'
 'wrinkles' 'black' 'white' 'tan' 'redness' 'light_coverage'
 'medium_coverage' 'full_coverage' 'sheer_finish' 'glowy_finish'
 'matte_finish' 'expensive' 'inexpensive' 'skin_concerns'
 'comfortable_wear' 'easy_use' 'pilling' 'shade_range']
[0.265 0.16  0.11  0.145 0.32 ]
      product_link_id  predicted_score
294               311             3.44
763               800             3.36
390               409             3.34
399               418             3.34
387               406             3.30
389               408             3.29
715               751             3.28
813      

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    0.2s finished


In [11]:
from ortools.linear_solver import pywraplp
import pandas as pd

from scipy.optimize import linprog
import numpy as np
import pandas as pd

df = pd.read_csv('cleaned_makeup_products.csv')
product_info = df[['product_link_id', 'product_name', 'brand', 'price', 'category']].copy()
product_info.rename(columns={'category': 'category_name'}, inplace=True)

# Merge data to include product prices and predicted scores
product_budget = product_copy.copy()
product_budget = pd.merge(product_budget, product_info, how='inner', on='product_link_id')


product_budget['category_name'].fillna(value='Uncategorized', inplace=True)
all_categories = []



skin_categories = ['Foundation', 'Tinted Moisturizer', 'BB & CC Creams']
allowed_skin_categories = []
allowed_makeup_products = []
if user_profile['light_coverage_review'] == 1: 
    allowed_makeup_products = ['Blush', 'Concealer', 'Makeup Remover', 'Setting Spray & Powder']
    if user_profile['easy_use_review'] == 1:
        allowed_skin_categories = ['Foundation', 'Tinted Moisturizer']
    elif user_profile['acne_review'] == 1:
        allowed_skin_categories = ['Foundation', 'BB & CC Creams']
    elif user_profile['comfortable_wear_review'] == 1:
        allowed_skin_categories = ['Tinted Moisturizer', 'BB & CC Creams']
    else: 
        allowed_skin_categories = ['Foundation', 'Tinted Moisturizer', 'BB & CC Creams']
elif user_profile['medium_coverage_review'] == 1: 
    allowed_makeup_products = ['Face Primer', 'Blush','Bronzer',  'Concealer', 'Makeup Remover', 'Setting Spray & Powder']
    allowed_skin_categories = ['Foundation', 'BB & CC Creams']
elif user_profile['full_coverage_review']: 
    allowed_makeup_products = ['Face Primer', 'Blush','Bronzer', 'Contouring', 'Highlighter', 
                               'Color Correcting',   
                               'Concealer', 'Makeup Remover', 'Setting Spray & Powder']
    allowed_skin_categories = ['Foundation']
        


# Parameters
B = 100  # Budget

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    raise Exception("Solver not created.")

# Create variables
product_vars = {}
for i, row in product_budget.iterrows():
    product_vars[i] = solver.BoolVar(f'product_{row["product_link_id"]}')

# Create constraints
# Budget constraint
budget_constraints = []
for i, row in product_budget.iterrows():
    budget_constraint = solver.Constraint(0, B)
    budget_constraint.SetCoefficient(product_vars[i], row['price'])
    budget_constraints.append(budget_constraint)

# Category constraint
categories = product_budget['category_name'].unique()
cat_constraints = []
allowed_skin_constraint = solver.Constraint(0, 1)
disallowed_constraint = solver.Constraint(0, 0)

print(allowed_skin_categories)
for cat in allowed_skin_categories:
    cat_df = product_budget[product_budget['category_name'] == cat]
    for i, row in cat_df.iterrows():
        allowed_skin_constraint.SetCoefficient(product_vars[i], 1)

for cat in skin_categories:
    if cat in allowed_skin_categories: continue
    cat_df = product_budget[product_budget['category_name'] == cat]
    for i, row in cat_df.iterrows():
        disallowed_constraint.SetCoefficient(product_vars[i], 1)

for cat in categories:
    if cat in allowed_makeup_products: continue
    if cat in allowed_skin_categories: continue
    cat_df = product_budget[product_budget['category_name'] == cat]
    for i, row in cat_df.iterrows():
        disallowed_constraint.SetCoefficient(product_vars[i], 1)
            
for cat in categories:
    if cat in skin_categories: continue
    if cat not in allowed_makeup_products: continue
    cat_constraint = solver.Constraint(0, 1)
    cat_df = product_budget[product_budget['category_name'] == cat]
    for i, row in cat_df.iterrows():
        cat_constraint.SetCoefficient(product_vars[i], 1)
    cat_constraints.append(cat_constraint)

# Objective function
objective = solver.Objective()
for i, row in product_budget.iterrows():
    objective.SetCoefficient(product_vars[i], row['predicted_score'])
objective.SetMaximization()

# Solve the problem
status = solver.Solve()

# Check results
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    for i, row in product_budget.iterrows():
        if product_vars[i].solution_value() == 1:
            print(f"category: {row['category_name']} ID: {row['product_link_id']} Product {row['product_name']} selected: Value Score = {row['predicted_score']}, Price = {row['price']}")
else:
    print('The problem does not have an optimal solution.')


/tmp/ipykernel_138885/4037537358.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  product_budget['category_name'].fillna(value='Uncategorized', inplace=True)


['Foundation', 'Tinted Moisturizer']
Solution:
category: Foundation ID: 137 Product 16h Cover & Last Powder Foundation selected: Value Score = 3.21, Price = 5.99
category: Concealer ID: 311 Product Futurist Soft Touch Brightening Skincealer Concealer selected: Value Score = 3.4400000000000004, Price = 38.0
category: Blush ID: 800 Product NUDIES MATTE LUX All Over Face Blush Color selected: Value Score = 3.36, Price = 35.0
category: Setting Spray & Powder ID: 1384 Product Vitamin Enriched Pressed Powder selected: Value Score = 3.27, Price = 53.0
category: Makeup Remover ID: 1444 Product Whoopsie Eye Liner Corrector selected: Value Score = 3.1950000000000003, Price = 10.0
